In [ ]:
# 📦 1. Install required dependencies
!pip install -q "dvc[gdrive]" transformers datasets mlflow seqeval
!pip install -q git+https://github.com/huggingface/transformers.git

In [ ]:
# 🧠 2. Clone the repo from the correct branch
!git clone --branch task-3 https://github.com/Teshager21/amharic-telegram-ecommerce-ner.git
%cd amharic-telegram-ecommerce-ner


In [ ]:
# 🔐 3. Upload service account key and configure DVC remote
from google.colab import files
import shutil, os
from pathlib import Path

# Upload the file
uploaded = files.upload()
filename = next(iter(uploaded))  # get the uploaded filename

# Move it to a safe path (avoid special characters)
key_path = Path("/content/sa_key.json")
shutil.move(filename, key_path)

# Configure DVC to use the uploaded key
!dvc remote modify gdrive_remote gdrive_use_service_account true
!dvc remote modify gdrive_remote gdrive_service_account_json_file_path {key_path}

In [ ]:
# ✅ 4. Pull data using DVC (with service account)
import subprocess

def dvc_pull_with_feedback():
    print("📥 Pulling DVC-tracked data from Google Drive remote...")
    result = subprocess.run(["dvc", "pull", "-v"], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ DVC pull successful.")
    else:
        print("❌ DVC pull failed:")
        print(result.stderr)

dvc_pull_with_feedback()


In [ ]:
# !dvc remote add -d gdrive_remote gdrive://1UveBgDaVcNQi1T-fLA-Kz03hPRqJ_ayk

In [ ]:
# # 1. Upload the service account JSON key file
# from google.colab import files
# import os
# import shutil
# from pathlib import Path

# uploaded = files.upload()
# filename = next(iter(uploaded))  # Get the uploaded filename
# src = Path("/content") / filename

# # 2. Move to your repo’s secrets folder
# repo_secrets_path = Path("/content/amharic-telegram-ecommerce-ner/secrets")
# repo_secrets_path.mkdir(parents=True, exist_ok=True)

# dest = repo_secrets_path / "dvc-sa-key.json"
# shutil.move(str(src), str(dest))

# print(f"✅ Key moved to repo secrets folder at: {dest}")

In [ ]:
# # 🔍 Check DVC remote config before pulling data
# !dvc remote list

# # Optionally, check detailed config
# !cat .dvc/config

# # Pull data and model if remote exists
# import subprocess

# def dvc_pull_if_remote_exists(remote_name="gdrive_remote"):
#     remotes = subprocess.getoutput("dvc remote list").splitlines()
#     remote_names = [line.split()[0] for line in remotes]
#     if remote_name in remote_names:
#         print(f"✅ Found remote '{remote_name}'. Pulling data...")

#         env = {**os.environ, "DVC_GDRIVE_USE_SERVICE_ACCOUNT": "true"}
#         result = subprocess.run(
#             ["dvc", "pull", "-v"],  # verbose
#             env=env,
#             capture_output=True,
#             text=True,
#         )

#         print(result.stdout)
#         if result.returncode != 0:
#             print("❌ Error during dvc pull:")
#             print(result.stderr)
#     else:
#         print(f"❌ Remote '{remote_name}' not found.")


# import os
# dvc_pull_if_remote_exists()

In [1]:
# Colab cell to run your fine-tuning main.py with Hydra config overrides

!python src/train/main.py \
  model.name_or_path=xlm-roberta-base \
  training.epochs=3 \
  training.batch_size=16 \
  data.conll_file=data/labeled/train.conll \
  output_dir=models/ner \
  logging.level=INFO


python: can't open file '/home/teshager/Documents/10Academy/repositories/projects/amharic-telegram-ecommerce-ner/notebooks/training/src/train/main.py': [Errno 2] No such file or directory


In [ ]:
# 🧠 Evaluate NER Model
from src.eval.evaluate import evaluate

LABEL_LIST = [
    "O",
    "B-PRODUCT", "I-PRODUCT",
    "B-PRICE", "I-PRICE",
    "B-LOC", "I-LOC",
]

In [ ]:
# Paths for model and test data
MODEL_PATH = "models/ner"  # Path to your fine-tuned model directory
TEST_DATA_PATH = "data/processed/ner_test.conll"  # Path to your test dataset in CoNLL format


In [ ]:
# Run evaluation
try:
    evaluate(
        model_path=MODEL_PATH,
        data_path=TEST_DATA_PATH,
        label_list=LABEL_LIST
    )
except Exception as e:
    print(f"Evaluation failed: {e}")


In [ ]:
# Launch MLflow UI in Colab
!mlflow ui --port 5000 &

# To open the UI, click the link below after running this cell:
print("MLflow UI should be running on port 5000")


In [ ]:
# 📊 View MLflow Run (Optional)
!ls -lh mlruns/